In [ ]:
from TorCtl import TorCtl
import urllib2

import socks
import time

import socket
socks.setdefaultproxy(socks.PROXY_TYPE_SOCKS5, "localhost", 9050)
socket.socket = socks.socksocket
import urllib2
from pcapfile import savefile
import pcapng
from pcapng import FileScanner
from pcapfile.protocols.linklayer import ethernet
from pcapfile.protocols.network import ip
import binascii
import pyshark
import json
source='172.21.64.145' # Change this to your computers IP address
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent}
 
def request(url):
    def _set_urlproxy():
        proxy_support = urllib2.ProxyHandler({"http" : "127.0.0.1:8118"})
        opener = urllib2.build_opener(proxy_support)
        urllib2.install_opener(opener)
    _set_urlproxy()
    request=urllib2.Request(url, None, headers)
    return urllib2.urlopen(request).read()
 
def renew_connection():
    conn = TorCtl.connect(controlAddr="127.0.0.1", controlPort=9050, passphrase="Simba_123")
    conn.send_signal("NEWNYM")
    conn.close()

    
ls=['apple','google','microsoft','cnn','gmail','yahoo','slack','wsj','quora','facebook','wikileaks','nytimes','']
for i in range(0,len(ls)):
    ls[i]="https://www."+ls[i]+".com"
    
    
def getnam(i):
    return i[i.find("www.")+4:i.find(".com")]


proxy_support = urllib2.ProxyHandler({"http" : "localhost:9050"})
opener = urllib2.build_opener(proxy_support) 
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
def proc(cp):
    l=[]
    cnt=0
    base=float(cp[0].sniff_timestamp)
    #print base
    for j in range(0,len(cp)):
        i=cp[j]
        if len(l)>=100:
            break
        try:
            src=i['ip'].src
            flag=1 if src==source else -1
            tm=float(i.sniff_timestamp)-base
            #print float(i.sniff_timestamp),base
            #print tm
            l.append(str(tm*flag))
            if len(l)>=100:
                break
        except:
            #print "Exception"
            continue
    return l

from multiprocessing import Process
global cap
import os

#cap = pyshark.LiveCapture(interface='en0')

def func2(nam):
    q= opener.open(nam)
    opener.close()
    
    


In [ ]:
for name in ls:
    cnt=0
    print name
    proxy_support = urllib2.ProxyHandler({"http" : "localhost:9050"})
    opener = urllib2.build_opener(proxy_support) 
    opener.addheaders = [('User-agent', 'Mozilla/5.0')]
    for i in range(0,200):
        print i
        cap = pyshark.LiveCapture(interface='en0') #interface is the medium used for connecting to via Tor
        #p1= Process(target=func1,args=(cap,))
        #p1.start()
        p2= Process(target=func2,args=(name,))
        p2.start()
        cap.sniff(timeout=3)
        cap.close()
        #cap = pyshark.LiveCapture(interface='en0')#, bpf_filter='ip and tcp port 80')
        #cap.sniff(timeout=5)
        #q= opener.open(name)#.read()
        #p1.join()
        p2.join()
        
        #time.sleep(5000)
        #p1.poll()
        #p1.wait()
        #os.wait(p1.pid())
        ws=proc(cap)
        json.dump(ws,open(getnam(name)+"_"+str(i)+".json",'wb'))